# Tasa de Servicio 


Cáulculo de la Tasa de Servicio para los proveedores CKD para su cumplimiento en la Expedición (AVIEXP), el Embarque (ETD) y la Llegada a puerto (ETA).

## Importar Librerias que se van a usar

In [1]:
#Importar Librerías
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os

## Cargar Archivos de Trabajo


* Archivos descargados desde SAP mediante la transacción VTT
* Archivo con los datos de la VTT de cada sourcing

1. Definir la ruta de los archivos

In [2]:
#Define the path
path=('C:/Users/Siberio/Desktop/Codigo TS/CSV/')
file_list = os.listdir(path)

#load the file with VTT info for each sourcing 
df_sourcings=pd.read_excel('C:/Users/Siberio/Desktop/Codigo TS/VTT FICHA.xlsx')

2. Combinar los archivos semanales

In [3]:
#Create a list for weekly files
dataframes = [] # Create a empty list
header_added = False  # To track if the header from the first file has been added

#Add all file names to a list
for file in file_list:
    if file.endswith('.csv'):
        file_path = os.path.join(path, file)
        df = pd.read_csv(file_path,sep=';',low_memory=False)
    
    #Add header of first file
    if not header_added:
        dataframes.append(df)  # Append the first DataFrame with header
        header_added = True
    else:
        dataframes.append(df.iloc[1:])  # Append subsequent DataFrames without header


# Concatenate all dataframes into a single dataframe
df_datos = pd.concat(dataframes, ignore_index=True)

3. Filtrar la información, dejando solo las columnas necesarias y eliminando las filas repetidas

In [4]:

#Delete unused columns
df_datos.drop(df_datos.columns[[0,1,3,6,7,8,9,10,11,12,13,14,15,16,17,18,20,21,22,23,24,25,26,27,30,31,33,34,35,36,37,38,39]],axis=1,inplace=True)
df_datos.drop(df_datos.columns[7],axis=1,inplace=True)

#Delete Suplementary RAN
df_datos = df_datos[df_datos['SemRAN'].str.contains('S') == False]



#Eliminar las Ran repetidas, conservando la que mas informacion contiene

# Define a function to select the row with the least number of null values
def select_row_least_nulls(group):
    null_counts = group.isnull().sum(axis=1)
    min_nulls = null_counts.min()
    return group[null_counts == min_nulls].iloc[0]

# Drop duplicates keeping the row with the least null values
df_datos = df_datos.groupby('RAN', group_keys=False).apply(select_row_least_nulls).reset_index(drop=True)

#Delete RAN´s without AVIEXP
df_datos = df_datos.dropna(subset=['F.Real Exp'])

#Combinar los dataframes de informacion de SAP y el archivo con informacion de los sourcings
df_datos = pd.merge(df_datos, df_sourcings, on ='Proveedor', how ='inner')

## Cálculos

1. Calcular la última semana del año anterior al año en curso, para tener en cuenta años de 52 y 53 semanas

In [5]:
#Calculate the last week of year-1
def get_last_iso_week(year):
    year=int(year)
    last_day = datetime(year, 12, 31)
    weekday = last_day.weekday()
    days_to_subtract = (weekday - 3) % 7
    last_iso_week_start = last_day - timedelta(days=days_to_subtract)
    
    # Calcula el número de la semana ISO
    iso_week_number = last_iso_week_start.isocalendar()[1]
    return iso_week_number

current_year=datetime.now().year
past_year=current_year-1
last_iso_week_number = get_last_iso_week(past_year)

2. Calcular semanas según parametros VTT para las 3 tasas

In [6]:
#Calcular semana real AVIEXP
df_datos['F.Real Exp'] = pd.to_datetime(df_datos['F.Real Exp'], errors ='coerce',dayfirst=True)
df_datos['F.Real Exp'].astype('int64').dtypes

#Calcular semana real Embarque
df_datos['F.Real Emb'] = pd.to_datetime(df_datos['F.Real Emb'], errors ='coerce',dayfirst=True)
df_datos['F.Real Emb'].astype('int64').dtypes

#Calcular semana real Puerto
df_datos['ETA'] = pd.to_datetime(df_datos['ETA'], errors ='coerce',dayfirst=True)
df_datos['ETA'].astype('int64').dtypes

#Tomar solo el numero de la semana ran
df_datos['Numero S.Ran']=df_datos['SemRAN'].str[1:3].astype(float)

#Calculo Semana en la cual se va a considerar en el calculo de la Tasa
df_datos['Semana Tasa AVIEXP']= df_datos['Numero S.Ran']-df_datos['RAN FINAL AVIEXP']
df_datos['Semana Tasa ETD']=df_datos['Numero S.Ran']-df_datos['RAN ETD']
df_datos['Semana Tasa ETA']=df_datos['Numero S.Ran']-df_datos['RAN FINAL ETA']

3. Calcular días según parametros VTT para las 3 tasas

In [7]:
#Funcion para calcular una fecha ingresando el año, la semana y el día de la semana (1 lunes-7 domingo)
def find_date_by_iso_week(year, iso_week, weekday):
    if iso_week < 1 or iso_week > 53 or weekday < 1 or weekday > 7:
        raise ValueError('El número de semana ISO debe estar entre 1 y 53, y el número de día de la semana debe estar entre 1 y 7.')
    
    date = datetime(year, 1, 1)
    days_difference = weekday - date.isoweekday() % 7
    if days_difference < 0:
        days_difference += 7
    
    target_date = date + timedelta(days=days_difference + (iso_week - 1) * 7)
    return target_date

4. Calcular Status AVIEXP

In [8]:
def Calculate_Status_Aviexp(row):
    year=datetime.now().year
    sem_aviexp_estandar_i=row['Numero S.Ran']-row['RAN INICIAL AVIEXP']
    sem_aviexp_estandar_f=row['Numero S.Ran']-row['RAN FINAL AVIEXP']

#Calcular semana estandar en caso de que cambie de año
    
    if sem_aviexp_estandar_i<0:
        year=year-1
    if sem_aviexp_estandar_i <0:
        sem_aviexp_estandar_i=last_iso_week_number-abs(sem_aviexp_estandar_i)
    elif sem_aviexp_estandar_i ==0:
        sem_aviexp_estandar_i=last_iso_week_number

    if sem_aviexp_estandar_f <0:
        sem_aviexp_estandar_f=last_iso_week_number-abs(sem_aviexp_estandar_f)
    elif sem_aviexp_estandar_f ==0:
        sem_aviexp_estandar_f=last_iso_week_number


    target_date_i= find_date_by_iso_week(year, sem_aviexp_estandar_i, row['DIA INICIAL EXP']) #calcular fecha inicial AVIEXP segun VTT
    target_date_f= find_date_by_iso_week(year, sem_aviexp_estandar_f, row['DIA FINAL EXP']) #calcular fecha final AVIEXP segun VTT


    #Calcular status del Aviexp, en caso de que falten datos ejecuta la excepcion
    try:
        if row['F.Real Exp']>=target_date_i and row['F.Real Exp']<=target_date_f: 
            status_AVIEXP='A TIEMPO'  #Si la fecha de Aviexp está en el rango de la VTT
        elif row['F.Real Exp']>target_date_f: 
            status_AVIEXP='EN ATRASO'  #Si la fecha de Aviexp es posterior al rango de la VTT
        else:
            status_AVIEXP='EN ADELANTO'  #Si la fecha de Aviexp es anteior al rango de la VTT
    except:
         status_AVIEXP='SIN DATOS'  #Si no hay datos para calcular

    return status_AVIEXP

#Crear columna Status AVIEXP y aplicar en cada fila la funcion de calculo del status        
df_datos['Status AVIEXP']=df_datos.apply(Calculate_Status_Aviexp,axis=1)

5. Calcular Status Embarque

In [9]:
# Funcion para calcular Status de Embarque
def Calculate_Status_Embarque(row):
    year=datetime.now().year
    sem_embarque_estandar=row['Numero S.Ran']-row['RAN ETD']
    

#Calcular semana estandar en caso de que cambie de año
    if sem_embarque_estandar<0:
        year=year-1

    if sem_embarque_estandar <0:
        sem_embarque_estandar=last_iso_week_number-abs(sem_embarque_estandar)
    elif sem_embarque_estandar ==0:
        sem_embarque_estandar=last_iso_week_number

    


    target_date= find_date_by_iso_week(year, sem_embarque_estandar, row['DIA ETD']) #calcular fecha ETD segun VTT


    #Calcular status del Aviexp, en caso de que falten datos ejecuta la excepcion
    try:
        if row['F.Real Emb']==target_date: 
            status_Embarque='A TIEMPO'  #Si la fecha de Emabrque es igual a la VTT
        elif row['F.Real Emb']>target_date: 
            status_Embarque='EN ATRASO'  #Si la fecha de Emabrque es posterior a la la VTT
        else:
            status_Embarque='EN ADELANTO'  #Si la fecha de Emabrque es anteior a la VTT
    except:
         status_Embarque='SIN DATOS'  #Si no hay datos para calcular

    return status_Embarque


#Crear columna Status ETD y aplicar en cada fila la funcion de calculo del status        
df_datos['Status ETD']=df_datos.apply(Calculate_Status_Embarque,axis=1)


6. Calcular Status Llegada

In [10]:

# Funcion para calcular Status de Llegada a Puerto
def Calculate_Status_Llegada(row):
    year=datetime.now().year
    sem_ETA_estandar_i=row['Numero S.Ran']-row['RAN INICIAL ETA']
    sem_ETA_estandar_f=row['Numero S.Ran']-row['RAN FINAL ETA']

#Calcular semana estandar en caso de que cambie de año
    
    if sem_ETA_estandar_i<0:
        year=year-1

    if sem_ETA_estandar_i <0:
        sem_ETA_estandar_i=last_iso_week_number-abs(sem_ETA_estandar_i)
    elif sem_ETA_estandar_i ==0:
        sem_ETA_estandar_i=last_iso_week_number

    if sem_ETA_estandar_f <0:
        sem_ETA_estandar_f=last_iso_week_number-abs(sem_ETA_estandar_f)
    elif sem_ETA_estandar_f ==0:
        sem_ETA_estandar_f=last_iso_week_number


    target_date_i= find_date_by_iso_week(year, sem_ETA_estandar_i, row['DIA INICIAL ETA']) #calcular fecha inicial ETA segun VTT
    target_date_f= find_date_by_iso_week(year, sem_ETA_estandar_f, row['DIA FINAL ETA']) #calcular fecha final ETA segun VTT


    #Calcular status del ETA, en caso de que falten datos ejecuta la excepcion
    try:
        if row['ETA']>=target_date_i and row['ETA']<=target_date_f: 
            status_ETA='A TIEMPO'  #Si la fecha de ETA está en el rango de la VTT
        elif row['ETA']>target_date_f: 
            status_ETA='EN ATRASO'  #Si la fecha de ETA es posterior al rango de la VTT
        else:
            status_ETA='EN ADELANTO'  #Si la fecha de ETA es anteior al rango de la VTT
    except:
         status_ETA='SIN DATOS'  #Si no hay datos para calcular

    return status_ETA


df_datos['Status ETA']=df_datos.apply(Calculate_Status_Llegada,axis=1)

7. Filtrar por Sourcing y Semana a considerar dentro del cálculo de la Tasa de Servicio, calcular porcentaje de cumplimiento (ALEJANDRA)

## Exportar Data Frame

1. Cambiar las semanas negativas por semanas del año anterior

In [11]:
#Funcion para calcular la semana en caso de que cambie de año
def calculate_past_year(row,column,last_iso_week_number):
    year=datetime.now().year
    if int(row[column]) < 0:
        year=year-1

    if int(row[column]) <0:
        new_value=last_iso_week_number-abs(row[column])
    elif int(row[column]) ==0:
        new_value=last_iso_week_number
    else:
        new_value=row[column]

    return int(new_value)


# Convertir los valores negativos a semanas del año anterior
df_datos['Semana Tasa AVIEXP']=df_datos.apply(lambda row:calculate_past_year(row,'Semana Tasa AVIEXP',last_iso_week_number),axis=1)
df_datos['Semana Tasa ETD']=df_datos.apply(lambda row:calculate_past_year(row,'Semana Tasa ETD',last_iso_week_number),axis=1)
df_datos['Semana Tasa ETA']=df_datos.apply(lambda row:calculate_past_year(row,'Semana Tasa ETA',last_iso_week_number),axis=1)

2. Exportar archivo consolidado cen formato csv

In [12]:
#Exportar como csv el dataframe con toda la informacion
df_datos.to_csv('C:/Users/Siberio/Desktop/Codigo TS/Archivo_Final.csv',index=False,sep=';')
print("finished")

finished
